In [1]:
import requests
import pandas as pd
import io
import os
import mysql.connector
import hashlib
import pandas as pd
import numpy as np

In [2]:
earthquake_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.csv'

In [3]:
s=requests.get(earthquake_url).content
earthquake_df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [4]:
def create_hash_column(dataframe):
    """
    Create a hash column by combining all other columns in the DataFrame into a string
    and hashing it using SHA-256.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The DataFrame with an additional 'hash_column' containing the hash values.
    """
    # Combine all columns into a single string
    combined_string = dataframe.apply(lambda row: ''.join(map(str, row)), axis=1)

    # Create a new column with the hash values
    dataframe['hash_column'] = combined_string.apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

    return dataframe

In [5]:
earthquake_df = create_hash_column(earthquake_df)

In [19]:
# clean df Nan
earthquake_df = earthquake_df.fillna(0.00)

In [21]:
earthquake_df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hash_column
0,2023-09-25T21:48:06.532Z,44.461000,-115.195000,10.00,2.50,ml,16.0,63.0,0.68600,0.24,...,"33 km NW of Stanley, Idaho",earthquake,1.91,2.012,0.081000,20.0,reviewed,us,us,b2ce2734dab68d651834a201fcb3a657992790488c0524...
1,2023-09-25T18:10:11.669Z,34.070333,-97.432333,6.31,2.51,ml,77.0,46.0,0.00000,0.37,...,"10 km S of Wilson, Oklahoma",earthquake,0.00,0.900,0.240000,22.0,reviewed,ok,ok,dee88db0c958cc83035e359502cd4065a4229a9e81dede...
2,2023-09-25T17:03:19.290Z,17.988500,-66.854167,11.53,2.50,md,4.0,164.0,0.02737,0.18,...,"3 km SE of Palomas, Puerto Rico",earthquake,4.72,2.140,0.241869,3.0,reviewed,pr,pr,ddb22372ed76e72e315e8d6219998f0c076bcaa1951088...
3,2023-09-25T16:42:01.180Z,17.954833,-66.843833,12.06,2.47,md,7.0,207.0,0.03926,0.11,...,"5 km SSW of Indios, Puerto Rico",earthquake,0.75,0.510,0.180713,5.0,reviewed,pr,pr,730f40cfd688e751df74e18e3638b8331d2db4499746bf...
4,2023-09-25T16:00:12.820Z,19.333167,-155.143500,1.25,3.02,ml,48.0,91.0,0.00000,0.17,...,"14 km S of Fern Forest, Hawaii",earthquake,0.26,0.330,0.176328,42.0,reviewed,hv,hv,7d9c07dab1c042c8eae2aac1bff268899fdceb12342acb...


In [7]:
def pandas_to_mysql_type(pandas_type):
    """
    Convert a Pandas data type to its MySQL equivalent.
    
    Args:
        pandas_type (str): The Pandas data type as a string.

    Returns:
        str: The MySQL data type as a string.
    """
    if pandas_type == 'object':
        return 'TEXT'
    elif pandas_type == 'int64':
        return 'INT'
    elif pandas_type == 'float64':
        return 'FLOAT'
    elif pandas_type == 'datetime64':
        return 'DATETIME'
    elif pandas_type == 'bool':
        return 'BOOL'
    else:
        # Handle other cases or raise an error for unsupported types.
        raise ValueError(f"Unsupported Pandas type: {pandas_type}")

def convert_pandas_to_mysql_types(dataframe):
    """
    Convert Pandas column types to MySQL data types for a DataFrame.

    Args:
        dataframe (pd.DataFrame): The Pandas DataFrame to convert.

    Returns:
        dict: A dictionary mapping column names to MySQL data types.
    """
    mysql_types = {}
    for column_name, dtype in dataframe.dtypes.items():
        mysql_type = pandas_to_mysql_type(dtype.name)
        mysql_types[column_name] = mysql_type
    return mysql_types

In [8]:
mysql_column_types = convert_pandas_to_mysql_types(earthquake_df)

In [9]:
mysql_column_types

{'time': 'TEXT',
 'latitude': 'FLOAT',
 'longitude': 'FLOAT',
 'depth': 'FLOAT',
 'mag': 'FLOAT',
 'magType': 'TEXT',
 'nst': 'FLOAT',
 'gap': 'FLOAT',
 'dmin': 'FLOAT',
 'rms': 'FLOAT',
 'net': 'TEXT',
 'id': 'TEXT',
 'updated': 'TEXT',
 'place': 'TEXT',
 'type': 'TEXT',
 'horizontalError': 'FLOAT',
 'depthError': 'FLOAT',
 'magError': 'FLOAT',
 'magNst': 'FLOAT',
 'status': 'TEXT',
 'locationSource': 'TEXT',
 'magSource': 'TEXT',
 'hash_column': 'TEXT'}

In [10]:
#create table 
table_name = 'portfolio.earthquake_table'
create_table_sql = f"CREATE TABLE {table_name} ("

for column_name, mysql_type in mysql_column_types.items():
    create_table_sql += f"{column_name} {mysql_type}, "

# Remove the trailing comma and space
create_table_sql = create_table_sql[:-2]

create_table_sql += ");"

In [11]:
create_table_sql

'CREATE TABLE portfolio.earthquake_table (time TEXT, latitude FLOAT, longitude FLOAT, depth FLOAT, mag FLOAT, magType TEXT, nst FLOAT, gap FLOAT, dmin FLOAT, rms FLOAT, net TEXT, id TEXT, updated TEXT, place TEXT, type TEXT, horizontalError FLOAT, depthError FLOAT, magError FLOAT, magNst FLOAT, status TEXT, locationSource TEXT, magSource TEXT, hash_column TEXT);'

In [43]:
connection = mysql.connector.connect(
        host=os.environ["MYSQL_ENDPOINT"],
        user=os.environ["MYSQL_USER"],
        password=os.environ["MYSQL_PASS"]
    )

In [13]:
cursor = connection.cursor()

In [14]:
cursor.execute(create_table_sql)

In [15]:
columns = ', '.join(mysql_column_types.keys())
placeholders = ', '.join(['%s'] * len(mysql_column_types))
insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

In [17]:
cursor.close()

True

In [18]:
cursor = connection.cursor()

In [22]:
#insert into table
for index, row in earthquake_df.iterrows():
    values = [row[column] for column in mysql_column_types.keys()]
    print(values)
    cursor.execute(insert_sql, values)

['2023-09-25T21:48:06.532Z', 44.461, -115.195, 10.0, 2.5, 'ml', 16.0, 63.0, 0.686, 0.24, 'us', 'us7000ky20', '2023-09-25T22:16:12.040Z', '33 km NW of Stanley, Idaho', 'earthquake', 1.91, 2.012, 0.081, 20.0, 'reviewed', 'us', 'us', 'b2ce2734dab68d651834a201fcb3a657992790488c05245bb2040a2f1f40b305']
['2023-09-25T18:10:11.669Z', 34.07033333, -97.43233333, 6.31, 2.51, 'ml', 77.0, 46.0, 0.0, 0.37, 'ok', 'ok2023svfh', '2023-09-25T21:15:16.753Z', '10 km S of Wilson, Oklahoma', 'earthquake', 0.0, 0.9, 0.24, 22.0, 'reviewed', 'ok', 'ok', 'dee88db0c958cc83035e359502cd4065a4229a9e81dede20923a44aeb986a108']
['2023-09-25T17:03:19.290Z', 17.9885, -66.8541666666667, 11.53, 2.5, 'md', 4.0, 164.0, 0.02737, 0.18, 'pr', 'pr71425878', '2023-09-25T17:19:51.500Z', '3 km SE of Palomas, Puerto Rico', 'earthquake', 4.72, 2.14, 0.241869394901518, 3.0, 'reviewed', 'pr', 'pr', 'ddb22372ed76e72e315e8d6219998f0c076bcaa1951088ca536b920c89726d9d']
['2023-09-25T16:42:01.180Z', 17.9548333333333, -66.8438333333333, 12.0

In [23]:
connection.commit()

In [64]:
cursor.close()

True

Workflow for Upserting into DB

1 . Get current hash List

2. Filter on DB download from API

3. Create insert statments of Hases that ARE NOT currently in DB

4. Insert and Commit Rows

5. Close Cursor and end funtion.


In [76]:
#upsert
cursor = connection.cursor()

In [45]:
hash_sql = cursor.execute("select hash_column from portfolio.earthquake_table")

In [48]:
hash_result = cursor.fetchall()

In [49]:
hash_list = [row[0] for row in hash_result]

In [51]:
hash_list[0]

'b2ce2734dab68d651834a201fcb3a657992790488c05245bb2040a2f1f40b305'

In [52]:
s=requests.get(earthquake_url).content
earthquake_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
earthquake_df = create_hash_column(earthquake_df)

In [53]:
earthquake_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hash_column
0,2023-09-25T22:41:19.660Z,17.910833,-66.842167,11.0400,2.51,md,16.0,214.0,0.07298,0.08,...,"8 km SE of Maria Antonia, Puerto Rico",earthquake,0.660000,0.350000,0.123629,6.0,reviewed,pr,pr,cdd6186d9c4933d3c9621ebd93c3d396ae706f4cd20a52...
1,2023-09-25T22:40:38.110Z,17.898667,-66.842167,11.3600,2.48,md,24.0,219.0,0.08378,0.18,...,"10 km SSE of Maria Antonia, Puerto Rico",earthquake,0.530000,0.420000,0.120035,6.0,reviewed,pr,pr,7c36ef7eab38ec4faf888445cb0e789acb4eaf5ae2ca67...
2,2023-09-25T21:48:06.532Z,44.461000,-115.195000,10.0000,2.50,ml,16.0,63.0,0.68600,0.24,...,"33 km NW of Stanley, Idaho",earthquake,1.910000,2.012000,0.081000,20.0,reviewed,us,us,d95cdf4d815814fee3cecaf1292cab2da2e483f7628c06...
3,2023-09-25T18:10:11.669Z,34.070333,-97.432333,6.3100,2.51,ml,77.0,46.0,0.00000,0.37,...,"10 km S of Wilson, Oklahoma",earthquake,NaN,0.900000,0.240000,22.0,reviewed,ok,ok,dee88db0c958cc83035e359502cd4065a4229a9e81dede...
4,2023-09-25T17:03:19.290Z,17.988500,-66.854167,11.5300,2.50,md,4.0,164.0,0.02737,0.18,...,"3 km SE of Palomas, Puerto Rico",earthquake,4.720000,2.140000,0.241869,3.0,reviewed,pr,pr,ddb22372ed76e72e315e8d6219998f0c076bcaa1951088...
5,2023-09-25T16:42:01.180Z,17.954833,-66.843833,12.0600,2.47,md,7.0,207.0,0.03926,0.11,...,"5 km SSW of Indios, Puerto Rico",earthquake,0.750000,0.510000,0.180713,5.0,reviewed,pr,pr,730f40cfd688e751df74e18e3638b8331d2db4499746bf...
6,2023-09-25T16:00:12.820Z,19.333167,-155.143500,1.2500,3.02,ml,48.0,91.0,NaN,0.17,...,"14 km S of Fern Forest, Hawaii",earthquake,0.260000,0.330000,0.176328,42.0,reviewed,hv,hv,7d9c07dab1c042c8eae2aac1bff268899fdceb12342acb...
7,2023-09-25T15:48:49.340Z,-6.950000,130.002900,90.6210,4.70,mb,51.0,87.0,2.41400,0.67,...,Banda Sea,earthquake,9.020000,2.358000,0.089000,38.0,reviewed,us,us,7da198cbbf82c6e17bdddfcfd1594b05bdc873f6a6bebd...
8,2023-09-25T15:21:09.820Z,18.002667,-66.857833,9.2900,2.54,md,5.0,145.0,0.03421,0.10,...,"2 km SE of Palomas, Puerto Rico",earthquake,3.030000,1.900000,0.127270,2.0,reviewed,pr,pr,9d5263eb05e61833f32c8fc48f4eb7c67b26bc1b53af2d...
9,2023-09-25T13:31:51.937Z,47.735800,-128.443400,10.0000,4.10,mb,57.0,182.0,2.31600,0.85,...,off the coast of Washington,earthquake,9.090000,1.988000,0.076000,47.0,reviewed,us,us,55f66f756306fcc6f3d6924d9a4f32794f358490f0b2a8...


In [71]:
upsert_earthquake_df =  earthquake_df[~earthquake_df['hash_column'].isin(hash_list)]

In [72]:
upsert_earthquake_df = upsert_earthquake_df.fillna(0.00)

In [73]:
upsert_earthquake_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hash_column
3,2023-09-25T18:10:11.669Z,34.070333,-97.432333,6.3100,2.51,ml,77.0,46.0,0.00000,0.37,...,"10 km S of Wilson, Oklahoma",earthquake,0.000000,0.900000,0.240000,22.0,reviewed,ok,ok,dee88db0c958cc83035e359502cd4065a4229a9e81dede...
4,2023-09-25T17:03:19.290Z,17.988500,-66.854167,11.5300,2.50,md,4.0,164.0,0.02737,0.18,...,"3 km SE of Palomas, Puerto Rico",earthquake,4.720000,2.140000,0.241869,3.0,reviewed,pr,pr,ddb22372ed76e72e315e8d6219998f0c076bcaa1951088...
5,2023-09-25T16:42:01.180Z,17.954833,-66.843833,12.0600,2.47,md,7.0,207.0,0.03926,0.11,...,"5 km SSW of Indios, Puerto Rico",earthquake,0.750000,0.510000,0.180713,5.0,reviewed,pr,pr,730f40cfd688e751df74e18e3638b8331d2db4499746bf...
6,2023-09-25T16:00:12.820Z,19.333167,-155.143500,1.2500,3.02,ml,48.0,91.0,0.00000,0.17,...,"14 km S of Fern Forest, Hawaii",earthquake,0.260000,0.330000,0.176328,42.0,reviewed,hv,hv,7d9c07dab1c042c8eae2aac1bff268899fdceb12342acb...
7,2023-09-25T15:48:49.340Z,-6.950000,130.002900,90.6210,4.70,mb,51.0,87.0,2.41400,0.67,...,Banda Sea,earthquake,9.020000,2.358000,0.089000,38.0,reviewed,us,us,7da198cbbf82c6e17bdddfcfd1594b05bdc873f6a6bebd...
8,2023-09-25T15:21:09.820Z,18.002667,-66.857833,9.2900,2.54,md,5.0,145.0,0.03421,0.10,...,"2 km SE of Palomas, Puerto Rico",earthquake,3.030000,1.900000,0.127270,2.0,reviewed,pr,pr,9d5263eb05e61833f32c8fc48f4eb7c67b26bc1b53af2d...
10,2023-09-25T12:11:59.322Z,-6.715700,129.737900,168.6760,4.50,mb,37.0,97.0,2.18500,0.75,...,Banda Sea,earthquake,8.680000,7.830000,0.106000,26.0,reviewed,us,us,6b6522e49c15f70eec5380d06adfe912660c3aed9373e0...
12,2023-09-25T09:10:06.640Z,36.683334,-121.352669,3.8100,2.61,md,47.0,54.0,0.02458,0.19,...,"12 km SSW of Tres Pinos, CA",earthquake,0.270000,0.970000,0.150000,55.0,automatic,nc,nc,b5da3b6766b2022ef581cfab849cdd92656bd2bf944453...
15,2023-09-25T07:31:47.990Z,18.099333,-66.928333,17.3500,2.46,md,11.0,77.0,0.09745,0.28,...,"4 km NE of Sabana Grande, Puerto Rico",earthquake,0.710000,1.200000,0.111121,4.0,reviewed,pr,pr,a08edc43348864243be22a2d74ce810de41044c83df194...
16,2023-09-25T06:57:45.899Z,13.821700,-91.892800,35.0000,4.60,mb,129.0,166.0,2.05400,0.90,...,"52 km S of Champerico, Guatemala",earthquake,9.990000,1.954000,0.033000,280.0,reviewed,us,us,9ea11fa0c066b2b91a78f86ba0a3d793827f0307e80515...


In [74]:
columns = ', '.join(mysql_column_types.keys())
placeholders = ', '.join(['%s'] * len(mysql_column_types))
insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

In [77]:
for index, row in upsert_earthquake_df.iterrows():
    values = [row[column] for column in mysql_column_types.keys()]
    print(values)
    cursor.execute(insert_sql, values)

['2023-09-25T18:10:11.669Z', 34.07033333, -97.43233333, 6.31, 2.51, 'ml', 77.0, 46.0, 0.0, 0.37, 'ok', 'ok2023svfh', '2023-09-25T21:15:16.753Z', '10 km S of Wilson, Oklahoma', 'earthquake', 0.0, 0.9, 0.24, 22.0, 'reviewed', 'ok', 'ok', 'dee88db0c958cc83035e359502cd4065a4229a9e81dede20923a44aeb986a108']
['2023-09-25T17:03:19.290Z', 17.9885, -66.8541666666667, 11.53, 2.5, 'md', 4.0, 164.0, 0.02737, 0.18, 'pr', 'pr71425878', '2023-09-25T17:19:51.500Z', '3 km SE of Palomas, Puerto Rico', 'earthquake', 4.72, 2.14, 0.241869394901518, 3.0, 'reviewed', 'pr', 'pr', 'ddb22372ed76e72e315e8d6219998f0c076bcaa1951088ca536b920c89726d9d']
['2023-09-25T16:42:01.180Z', 17.9548333333333, -66.8438333333333, 12.06, 2.47, 'md', 7.0, 207.0, 0.03926, 0.11, 'pr', 'pr71425868', '2023-09-25T16:57:02.100Z', '5 km SSW of Indios, Puerto Rico', 'earthquake', 0.75, 0.51, 0.180713256236441, 5.0, 'reviewed', 'pr', 'pr', '730f40cfd688e751df74e18e3638b8331d2db4499746bf5ef53eabe5d3632eb6']
['2023-09-25T16:00:12.820Z', 19.

In [78]:
connection.commit()
cursor.close()

True